# Numerical Integration: Simpson Adaptive Integration

---

## Adaptive Simpson's method, also called adaptive Simpson's rule, is a method of numerical integration proposed by G.F. Kuncir in 1962.

<img src="http://terpconnect.umd.edu/~petersd/460/html/adapt_test_01.png" width=500>

## The idea of adaptive quadrature is largely motivated by the following question: Can we minimise the computational cost/function evaluation such that the approximation is within a given tolerance?

---
## How to implement this idea?

### Consider the function $f$ is continuous on the intervals $[a,\,b]$.
### Let $S(a,b)$ be the Simpson value on the interval $[a,\,b]$, then the error is $\left| S(a,b) - \left[ S(a, \frac{a + b}{2}) + S(\frac{a + b}{2}, b) \right] \right|$.

### If error $< \varepsilon$, then we can use the Simpson value to aprroximate the integral of $f$.
### $$\int_a^b\,f(x)\,dx \approx S(a,b)$$

### If NOT, then we divide the interval $[a,\,b]$ into the sub-intervals $\left[a, \frac{a + b}{2}\right]$ and $\left[\frac{a + b}{2}, b\right]$ to approximate the integral of $f$.
### $$\int_a^b\,f(x)\,dx \approx S(a, \frac{a + b}{2}) + S(\frac{a + b}{2}, b)$$
### Until the error is less than $\varepsilon$.

---
## Error Estimate

### Let $S_N(f)$ denote Simpson's rule
### $$S_N(f) = \frac{\Delta x}{6}\,\sum_{i=1}^{N}\,(f(x_{i - 1}) + 4f(\frac{x_i + x_{i - 1}}{2}) + f(x_{i}))$$
### where $N$ is a positive integer number of sub-intervals of $[a,\,b],\,\Delta x = \frac{b - a}{N}$ and $x_i = a + i \Delta x$.

### Let $T_N(f)$ denote trapezoid rule
### $$T_N(f) = \frac{\Delta x}{2} \sum_{i = 1}^N (f(x_i) + f(x_{i-1}))$$
### where $\Delta x = \frac{b - a}{N}$ is the length of the sub-intervals and $x_i = a + i\,\Delta x$.

### Then simpson formula resulting from one level of extrapolation from trapezoid estimates:
### $$S_{2N}(f) = \frac{1}{3}\left[4T_{2N}(f) - T_{N}(f) \right].$$

### We expect this method to be fourth-order accrate, i.e.,
### $$\int_a^b\,f(x)\,dx = S_{2N}(f) + O(N^{-4}),$$

### We can further extrapolate to six-order accuracy,
### $$R_{4N}(f) = \frac{1}{15} \left[ 16 S_{4N}(f) - S_{2N}(f) \right].$$

### By virtue of higher order of accuracy, $R_{4N}(f)$ should be more accurate than $S_{4N}(f)$.
### Hence a decent estimate of the error, $E$, in the better of the two Simpson values is
### $$E = R_{4N}(f) - S_{4N}(f) = \frac{S_{4N}(f) - S_{2N}(f)}{15}.$$

---
## Prerequisites

### 1. The integrand function $f(x)$.
### 2. The interval $[a,\,b]$.
### 3.  The acceptable error.

---
## Implementation

### Part 0. Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Part 1. Implement Simpson Adaptive Integration

In [ ]:
def simpson_adaptive(
    f,
    a,
    b,
    tolerance=1e-7
):
    
    '''
    Parameters
    ----------
    f : function
        Vectorized function of a single variable
    a , b : numbers
        Interval of integration [a,b]
    tolerance : float
        One of the termination conditions. Error tolerance.

    Returns
    -------
    S_adp : float
        Approximation of the integral of f(x) from a to b using Simpson Adaptive Integration.
    nodes : the sequence of float number
        All nodes we use to approximate the integral.
    '''
    
    m = (b + a) / 2
    
    def do_integral(
        a,
        f_of_a,
        b,
        f_of_b,
        m,
        f_of_m,
        tolerance
    ):
        # These are the two new nodes and their f values.
        x_left = (a + m) / 2
        f_of_x_left = f(x_left)
        x_right = (m + b) / 2
        f_of_x_right = f(x_right)
        
        # Record all nodes we use
        nodes = [a,  x_left, m, x_right, b]
        
        # Compute the trapezoid values iteratively.
        h = (b - a)
        Trapezoid = np.zeros(3, None)
        Trapezoid[0] = h * (f_of_a + f_of_b) / 2
        Trapezoid[1] = Trapezoid[0] / 2 + (h / 2) * f_of_m
        Trapezoid[2] = Trapezoid[1] / 2 + (h / 4) * (f_of_x_left + f_of_x_right)
        
        # Compute the value of Simpson's rule.
        S = (4 * Trapezoid[1:] - Trapezoid[0:2]) / 3
        
        # Compute error estimate.
        error = (S[1] - S[0]) / 15
        
        # Acceptable error
        if abs(error) < tolerance * (1 + abs(S[1])):
            return S[1], nodes
        
        # Error is too large--bisect and recurse.
        else:
            S_adp_left, nodes_left = do_integral(a, f_of_a, m, f_of_m, x_left, f_of_x_left, tolerance)
            S_adp_right, nodes_right = do_integral(m, f_of_m, b, f_of_b, x_right, f_of_x_right, tolerance)
            
            # Add the both sides of approximate values 
            S_adp = S_adp_left + S_adp_right
            
            # Collect all nodes from left and right
            nodes = sum([nodes_left, nodes_right[1:]], [])
            
            return S_adp, nodes
    
    S_adp, nodes = do_integral(a, f(a), b, f(b), m, f(m), tolerance)
    
    return S_adp, nodes

---
## Example

## 1. Compute the function on integrals for which we know the exact value.
### $$\int_0^{\pi/2}\,\sin x\,dx = 1$$

In [ ]:
approximation, nodes = simpson_adaptive(np.sin, 0, np.pi/2)
print('The approximate value is', approximation)
# print('All nodes are', nodes)
print('The number of nodes is', len(nodes))

### Error 

In [ ]:
error = abs(approximation - 1)
print('The error is', error)

## 2. Approximate  $\ln 2$
### Find a value $N$ which guarantees that Simpson adaptive approximation of the integral
### $$\int_1^2\,\frac{1}{x}\,dx$$

In [ ]:
approximation, nodes = simpson_adaptive(lambda x : 1/x, 1, 2, 0.0001)
print('The approximate value is', approximation)
print('All nodes are', nodes)
print('The number of nodes is', len(nodes))

### Error

In [ ]:
error = abs(approximation - np.log(2))
print('The error is', error)

# End